In [3]:
import json
import uuid
import boto3
import s3fs
import rasterio
import xarray as xr
import rioxarray
from pystac_client import Client
import IPython.display
from IPython.display import display_html, display_javascript
from copy import deepcopy
import geopandas as gpd
import pandas as pd
from shapely.geometry import shape
import hvplot.pandas  # noqa: F401

/home/apolo/anaconda3/envs/geospatial/lib/python3.11/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)


# CQL2 Filtering

In [4]:
# convert a list of STAC Items into a GeoDataFrame
def items_to_geodataframe(items):
    _items = []
    for i in items:
        _i = deepcopy(i)
        _i["geometry"] = shape(_i["geometry"])
        _items.append(_i)
    gdf = gpd.GeoDataFrame(pd.json_normalize(_items))
    for field in ["properties.datetime", "properties.created", "properties.updated"]:
        if field in gdf:
            gdf[field] = pd.to_datetime(gdf[field])
    gdf.set_index("properties.datetime", inplace=True)
    return gdf

In [5]:
# STAC API root URL
URL = "https://planetarycomputer.microsoft.com/api/stac/v1"

# custom headers
headers = []

cat = Client.open(URL, headers=headers)

### Initial Search Parameters

In [6]:
# AOI around Delfzijl, in the north of The Netherlands
geom = {
    "type": "Polygon",
    "coordinates": [
        [
            [6.42425537109375, 53.174765470134616],
            [7.344360351562499, 53.174765470134616],
            [7.344360351562499, 53.67393435835391],
            [6.42425537109375, 53.67393435835391],
            [6.42425537109375, 53.174765470134616],
        ]
    ],
}

params = {
    "max_items": 100,
    "collections": "landsat-8-c2-l2",
    "intersects": geom,
    "datetime": "2018-01-01/2020-12-31",
}


# reusable search function
def search_fetch_plot(params, filt):
    # limit sets the # of items per page so we can see multiple pages getting fetched
    params["filter"] = filt
    search = cat.search(**params)
    items = list(search.items_as_dicts())  # safe b/c we set max_items = 100
    # DataFrame
    items_df = pd.DataFrame(items_to_geodataframe(items))
    print(f"{len(items_df.index)} items found")
    field = "properties.eo:cloud_cover"
    return items_df.hvplot(
        y=field, label=json.dumps(filt), frame_height=500, frame_width=800
    )

### CQL2 Filters

In [7]:
filt = {"op": "lte", "args": [{"property": "eo:cloud_cover"}, 10]}

search_fetch_plot(params, filt)

33 items found


AttributeError: 'DataFrame' object has no attribute 'hvplot'

In [ ]:
filt = {"op": "gte", "args": [{"property": "eo:cloud_cover"}, 80]}

search_fetch_plot(params, filt)

In [ ]:
filt = {
    "op": "and",
    "args": [
        {"op": "lte", "args": [{"property": "eo:cloud_cover"}, 60]},
        {"op": "gte", "args": [{"property": "eo:cloud_cover"}, 40]},
    ],
}

search_fetch_plot(params, filt)